In [15]:
from sqlalchemy import create_engine, inspect
import sqlite3

class DatabaseAnalyzer:
    def __init__(self, db_path):
        self.db_path = db_path
        self.engine = create_engine(f"sqlite:///{db_path}")
        self.inspector = inspect(self.engine)
    
    def get_schema_summary(self):
        """Extract schema details (tables, columns, PKs, FKs) for reasoning."""
        schema = {}
        for table in self.inspector.get_table_names():
            columns = []
            for col in self.inspector.get_columns(table):
                col_info = {
                    "name": col["name"],
                    "type": str(col["type"]),  # Convert SQLAlchemy type to string
                    "nullable": col["nullable"],
                    "default": col["default"]
                }
                columns.append(col_info)
            
            # Get primary keys (handle empty cases)
            try:
                pks = self.inspector.get_pk_constraint(table)["constrained_columns"]
            except:
                pks = []
            
            # Get foreign keys
            fks = []
            try:
                for fk in self.inspector.get_foreign_keys(table):
                    fks.append({
                        "column": fk["constrained_columns"][0],
                        "references": f"{fk['referred_table']}({fk['referred_columns'][0]})"
                    })
            except:
                pass
            
            schema[table] = {
                "columns": columns,
                "primary_keys": pks,
                "foreign_keys": fks
            }
        return schema

# Example usage:
analyzer = DatabaseAnalyzer("spider_sqlite/activity_1/activity_1.sqlite")
print(analyzer.get_schema_summary())

{'activity': {'columns': [{'name': 'activity_id', 'type': 'NUMERIC', 'nullable': True, 'default': None}, {'name': 'activity_name', 'type': 'TEXT', 'nullable': True, 'default': None}], 'primary_keys': ['activity_id'], 'foreign_keys': []}, 'faculty': {'columns': [{'name': 'faculty_id', 'type': 'NUMERIC', 'nullable': True, 'default': None}, {'name': 'last_name', 'type': 'TEXT', 'nullable': True, 'default': None}, {'name': 'first_name', 'type': 'TEXT', 'nullable': True, 'default': None}, {'name': 'rank', 'type': 'TEXT', 'nullable': True, 'default': None}, {'name': 'sex', 'type': 'TEXT', 'nullable': True, 'default': None}, {'name': 'phone', 'type': 'NUMERIC', 'nullable': True, 'default': None}, {'name': 'room', 'type': 'TEXT', 'nullable': True, 'default': None}, {'name': 'building', 'type': 'TEXT', 'nullable': True, 'default': None}], 'primary_keys': ['faculty_id'], 'foreign_keys': []}, 'faculty_participates_in': {'columns': [{'name': 'faculty_id', 'type': 'NUMERIC', 'nullable': True, 'defa

In [16]:
def execute_sql_with_context(db_path, query, schema):
    """Execute SQL and return results with metadata (column types, row counts)."""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Execute query
        cursor.execute(query)
        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description] if cursor.description else []
        
        # Generate metadata
        metadata = {
            "row_count": len(results),
            "columns": columns,
            "column_types": self._infer_column_types(results, columns, schema) if results else []
        }
        
        conn.close()
        return {
            "success": True,
            "results": results,
            "metadata": metadata
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "suggestion": self._suggest_fix(query, schema)
        }

def _infer_column_types(self, results, columns, schema):
    """Infer column types from data (fallback to schema if empty results)."""
    if not results:
        return [{"name": col, "type": "unknown"} for col in columns]
    
    # Sample first row for type inference
    sample = results[0]
    types = []
    for i, val in enumerate(sample):
        col_name = columns[i]
        types.append({
            "name": col_name,
            "type": type(val).__name__,
            "sample_value": str(val)[:50]  # Truncate for display
        })
    return types

In [17]:
def optimize_and_validate(query, schema):
    """Check for common issues (missing JOINs, inefficient filters)."""
    parsed = sqlparse.parse(query)[0]
    tokens = [t for t in parsed.flatten() if t.ttype not in sqlparse.tokens.Whitespace]
    
    warnings = []
    
    # Check for Cartesian products
    if "JOIN" not in query.upper() and len([t for t in schema.keys() if t in query]) > 1:
        warnings.append("WARNING: Possible Cartesian product detected (missing JOIN).")
    
    # Check for missing WHERE on large tables
    if "WHERE" not in query.upper() and any(t in query for t in ["customer", "order"]):
        warnings.append("WARNING: No WHERE clause on a large table (may be slow).")
    
    return {
        "is_valid": len(warnings) == 0,
        "warnings": warnings,
        "optimized_query": self._apply_optimizations(query, schema)
    }

def _apply_optimizations(self, query, schema):
    """Simple optimizations (e.g., adding LIMIT to exploratory queries)."""
    if "SELECT" in query.upper() and "LIMIT" not in query.upper():
        return query.rstrip(";") + " LIMIT 100;"
    return query

In [18]:
from typing import List, Dict
import re

class AmbiguityResolver:
    def __init__(self, schema: Dict):
        self.schema = schema
    
    def resolve(self, question: str, candidate_queries: List[str]) -> Dict:
        """Resolve ambiguity by analyzing candidate queries + schema."""
        scores = []
        for query in candidate_queries:
            score = 0
            
            # Score based on schema coverage
            tables_mentioned = re.findall(r"FROM\\s+(\\w+)", query, re.IGNORECASE)
            for table in tables_mentioned:
                if table in self.schema:
                    score += 10 * len(self.schema[table]["columns"])
            
            # Penalize overly broad queries
            if "WHERE" not in query:
                score -= 20
            
            scores.append(score)
        
        best_idx = scores.index(max(scores))
        return {
            "selected_query": candidate_queries[best_idx],
            "alternatives": [q for i, q in enumerate(candidate_queries) if i != best_idx],
            "confidence": min(100, max(0, scores[best_idx] // 2))  # Scale to 0-100
        }

In [19]:
def interactive_text_to_sql_enhanced(db_path=None, max_questions=3):
    analyzer = DatabaseAnalyzer(db_path) if db_path else None
    resolver = AmbiguityResolver(analyzer.get_schema_summary()) if analyzer else None
    
    for i in range(max_questions):
        question = input(f"\\nQuestion {i+1}/{max_questions}: ").strip()
        if question.lower() in ["exit", "quit"]:
            break
            
        # Generate multiple candidate queries
        candidates = self._generate_candidate_queries(question)
        
        # Resolve ambiguity
        if resolver and len(candidates) > 1:
            resolution = resolver.resolve(question, candidates)
            print(f"\\n[Agent] Selected query (confidence: {resolution['confidence']}%):")
            print(resolution["selected_query"])
            if resolution["alternatives"]:
                print("\\nAlternatives considered:")
                for alt in resolution["alternatives"]:
                    print(f"- {alt}")
        else:
            resolution = {"selected_query": candidates[0]}
        
        # Execute and display
        if db_path:
            result = execute_sql_with_context(db_path, resolution["selected_query"], analyzer.schema)
            if result["success"]:
                print("\\nResults:")
                print(tabulate(result["results"], headers=result["metadata"]["columns"]))
            else:
                print(f"\\nError: {result['error']}")
                print(f"Suggestion: {result['suggestion']}")

def _generate_candidate_queries(self, question):
    """Generate 3 variants of the query using temperature sampling."""
    prompts = [f"Question: {question}\\nSQL Query {i+1}:" for i in range(3)]
    return [
        self._generate_sql(prompt)
        for prompt in prompts
    ]

In [32]:
import sqlite3
import sqlparse
import torch
import random
from tabulate import tabulate
from typing import Dict, List, Tuple, Optional
import re
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import json
import os
import datetime

class Text2SQLAgent:
    def __init__(self, model_path: str, db_path: str, populate_if_empty: bool = True):
        """
        Initialize the Text-to-SQL Agent.
        
        Args:
            model_path: Path to the directory containing the fine-tuned model files
            db_path: Path to the SQLite database
            populate_if_empty: If True, automatically populate tables if they're empty
        """
        self.model, self.tokenizer = self._load_model(model_path)
        self.db_path = db_path
        self.schema_cache = self._load_schema()
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)
        
        # Check if database is empty and populate if needed
        if populate_if_empty:
            self._check_and_populate_data()
        
        # Reload schema after potential population
        self.schema_cache = self._load_schema()
        
        # Log connection status
        print(f"Successfully connected to database: {db_path}")
        print(f"Loaded schema with {len(self.schema_cache)} tables")
    
    def _load_model(self, model_path: str):
        """Load the model and tokenizer from the specified path"""
        try:
            # Load tokenizer first
            tokenizer = AutoTokenizer.from_pretrained(model_path)
            
            # Load base model config from adapter config
            config = PeftConfig.from_pretrained(model_path)
            
            # Load base model
            base_model = AutoModelForCausalLM.from_pretrained(
                config.base_model_name_or_path,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map="auto"
            )
            
            # Load the PEFT adapter model
            model = PeftModel.from_pretrained(
                base_model,
                model_path,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
            )
            
            # Merge and unload the adapter to make inference faster
            model = model.merge_and_unload()
            
            return model, tokenizer
        except Exception as e:
            raise RuntimeError(f"Error loading model: {str(e)}")
    
    def _load_schema(self) -> Dict[str, List[Dict]]:
        """
        Load and cache the database schema.
        
        Returns:
            Dictionary with table names as keys and lists of column info as values.
        """
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            # Get all tables
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = [table[0] for table in cursor.fetchall()]
            
            schema = {}
            for table in tables:
                cursor.execute(f"PRAGMA table_info({table});")
                columns = [{"name": col[1], "type": col[2], "notnull": col[3], "pk": col[5]} 
                          for col in cursor.fetchall()]
                schema[table] = columns
                
                # Show table structure 
                print(f"Table '{table}' columns: {', '.join(col['name'] for col in columns)}")
                
                # Get row count and sample data
                try:
                    cursor.execute(f"SELECT COUNT(*) FROM {table}")
                    count = cursor.fetchone()[0]
                    print(f"Row count: {count}")
                    
                    if count > 0:
                        cursor.execute(f"SELECT * FROM {table} LIMIT 1")
                        sample = cursor.fetchone()
                        if sample:
                            print(f"Sample data: {sample}")
                except Exception as e:
                    print(f"Error getting sample data: {str(e)}")
            
            conn.close()
            return schema
        except Exception as e:
            print(f"Error loading schema: {str(e)}")
            return {}
    
    def _check_and_populate_data(self):
        """Check if tables are empty and populate with sample data if needed"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            # Check if any tables are empty
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = [table[0] for table in cursor.fetchall()]
            
            all_empty = True
            for table in tables:
                cursor.execute(f"SELECT COUNT(*) FROM {table}")
                count = cursor.fetchone()[0]
                if count > 0:
                    all_empty = False
                    break
            
            # If all tables are empty, populate with sample data
            if all_empty:
                print("\n--- DATABASE IS EMPTY. POPULATING WITH SAMPLE DATA ---\n")
                self._populate_database(conn, cursor)
                print("\n--- DATABASE POPULATION COMPLETE ---\n")
            
            conn.close()
        except Exception as e:
            print(f"Error checking/populating data: {str(e)}")
    
    def _populate_database(self, conn, cursor):
        """Populate the database with sample data"""
        try:
            # Create sample data for each table based on schema
            
            # 1. Populate activity table
            activities = [
                (1, "Chess Club"),
                (2, "Debate Team"),
                (3, "Basketball"),
                (4, "Science Olympiad"),
                (5, "Drama Club"),
                (6, "Student Council"),
                (7, "Math Club"),
                (8, "Robotics"),
                (9, "Music Band"),
                (10, "Art Club"),
                (11, "Coding Club"),
                (12, "Environmental Club"),
                (13, "Tennis"),
                (14, "Swimming"),
                (15, "Soccer")
            ]
            
            cursor.executemany("INSERT INTO activity (activity_id, activity_name) VALUES (?, ?)", activities)
            print(f"Inserted {len(activities)} activities")
            
            # 2. Populate faculty table
            first_names = ["James", "Mary", "John", "Patricia", "Robert", "Jennifer", "Michael", "Linda", 
                          "William", "Elizabeth", "David", "Susan", "Richard", "Jessica", "Joseph", 
                          "Sarah", "Thomas", "Karen", "Charles", "Nancy", "Daniel", "Lisa", "Matthew", 
                          "Margaret", "Anthony", "Sandra", "Mark", "Ashley", "Donald", "Kimberly"]
            
            last_names = ["Smith", "Johnson", "Williams", "Jones", "Brown", "Davis", "Miller", "Wilson", 
                         "Moore", "Taylor", "Anderson", "Thomas", "Jackson", "White", "Harris", "Martin", 
                         "Thompson", "Garcia", "Martinez", "Robinson", "Clark", "Rodriguez", "Lewis", 
                         "Lee", "Walker", "Hall", "Allen", "Young", "King", "Wright"]
            
            ranks = ["Professor", "Associate Professor", "Assistant Professor", "Instructor", "Adjunct"]
            buildings = ["Science Hall", "Thompson Building", "Arts Center", "Engineering Building", "Library"]
            
            faculty_data = []
            for i in range(1, 51):  # Create 50 faculty members
                faculty_data.append((
                    i,  # faculty_id
                    random.choice(last_names),  # last_name
                    random.choice(first_names),  # first_name
                    random.choice(ranks),  # rank
                    random.choice(['M', 'F']),  # sex
                    f"555{random.randint(1000000, 9999999)}",  # phone
                    f"{random.randint(100, 500)}",  # room
                    random.choice(buildings)  # building
                ))
            
            cursor.executemany("""
                INSERT INTO faculty (faculty_id, last_name, first_name, rank, sex, phone, room, building) 
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, faculty_data)
            print(f"Inserted {len(faculty_data)} faculty members")
            
            # 3. Populate student table
            majors = [101, 102, 103, 104, 105]  # Sample major codes
            city_codes = ["NYC", "LA", "CHI", "HOU", "PHX", "PHI", "SAN", "DAL", "ATL", "BOS"]
            
            student_data = []
            for i in range(1, 201):  # Create 200 students
                student_data.append((
                    i,  # student_id
                    random.choice(last_names),  # last_name
                    random.choice(first_names),  # first_name
                    random.randint(18, 30),  # age
                    random.choice(['M', 'F']),  # sex
                    random.choice(majors),  # major
                    random.randint(1, 50),  # advisor (faculty_id)
                    random.choice(city_codes)  # city_code
                ))
            
            cursor.executemany("""
                INSERT INTO student (student_id, last_name, first_name, age, sex, major, advisor, city_code) 
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, student_data)
            print(f"Inserted {len(student_data)} students")
            
            # 4. Populate faculty_participates_in table
            # Each activity will have varying number of faculty participants
            faculty_activities = []
            for activity_id in range(1, 16):
                # Assign 3-10 random faculty to each activity
                num_faculty = random.randint(3, 10)
                faculty_ids = random.sample(range(1, 51), num_faculty)
                
                for faculty_id in faculty_ids:
                    faculty_activities.append((faculty_id, activity_id))
            
            cursor.executemany("""
                INSERT INTO faculty_participates_in (faculty_id, activity_id) 
                VALUES (?, ?)
            """, faculty_activities)
            print(f"Inserted {len(faculty_activities)} faculty activity associations")
            
            # 5. Populate participates_in table
            # Each student participates in 1-3 activities
            student_activities = []
            for student_id in range(1, 201):
                # Assign 1-3 random activities to each student
                num_activities = random.randint(1, 3)
                activity_ids = random.sample(range(1, 16), num_activities)
                
                for activity_id in activity_ids:
                    student_activities.append((student_id, activity_id))
            
            cursor.executemany("""
                INSERT INTO participates_in (student_id, activity_id) 
                VALUES (?, ?)
            """, student_activities)
            print(f"Inserted {len(student_activities)} student activity associations")
            
            # Commit all changes
            conn.commit()
            
        except Exception as e:
            conn.rollback()
            print(f"Error during data population: {str(e)}")
    
    def _format_schema_prompt(self) -> str:
        """
        Format the schema as a string for the prompt.
        
        Returns:
            String representation of the schema.
        """
        schema_str = []
        for table, columns in self.schema_cache.items():
            cols = [f"{col['name']} ({col['type']})" for col in columns]
            schema_str.append(f"Table {table}: {', '.join(cols)}")
        return "\n".join(schema_str) if schema_str else "Schema not available"
    
    def _is_ambiguous(self, question: str) -> Tuple[bool, Optional[str]]:
        """
        Detect if a question is ambiguous based on keywords or schema analysis.
        
        Args:
            question: Natural language question
            
        Returns:
            Tuple of (is_ambiguous, clarification_message)
        """
        ambiguous_keywords = ['any', 'some', 'details', 'information']
        question_lower = question.lower()
        
        # Check for vague keywords
        if any(keyword in question_lower for keyword in ambiguous_keywords):
            return True, "The question is vague. Please specify which fields or conditions you want to query."
        
        # Extract words and schema elements
        words = re.findall(r'\b\w+\b', question_lower)
        schema_tables = set(table.lower() for table in self.schema_cache.keys())
        schema_columns = set(col['name'].lower() for table in self.schema_cache.values() 
                           for col in table)
        
        # Include singular forms of plural nouns (e.g., 'buildings' -> 'building')
        mentioned_tables = []
        mentioned_columns = []
        for word in words:
            if word in schema_tables:
                mentioned_tables.append(word)
            elif word in schema_columns:
                mentioned_columns.append(word)
            else:
                # Check singular form
                singular = word.rstrip('s')
                if singular in schema_tables:
                    mentioned_tables.append(singular)
                elif singular in schema_columns:
                    mentioned_columns.append(singular)
        
        # Check for aggregation keywords or specific terms like 'professors', 'rooms'
        aggregation_keywords = ['number', 'count', 'total', 'sum', 'average']
        has_aggregation = any(keyword in question_lower for keyword in aggregation_keywords)
        has_specific_terms = any(term in question_lower for term in ['professor', 'room', 'faculty', 'activity'])
        
        if not mentioned_tables and not mentioned_columns and not has_aggregation and not has_specific_terms:
            valid_tables = ", ".join(self.schema_cache.keys())
            clarification = (f"The question doesn't reference any specific tables or columns. "
                           f"Available tables: {valid_tables}. Please clarify.")
            return True, clarification
        
        return False, None
    
    def _optimize_query(self, sql: str) -> str:
        """
        Apply basic query optimization.
        
        Args:
            sql: Raw SQL query
            
        Returns:
            Optimized SQL query
        """
        try:
            parsed = sqlparse.parse(sql)[0]
            tokens = [t for t in parsed.tokens if not t.is_whitespace]
            
            # Avoid SELECT * by replacing with specific columns if possible
            if 'SELECT *' in sql.upper():
                from_clause = None
                for i, token in enumerate(tokens):
                    if token.normalized == 'FROM':
                        from_clause = tokens[i+1].value
                        break
                
                if from_clause in self.schema_cache:
                    columns = [col['name'] for col in self.schema_cache[from_clause]]
                    sql = sql.replace('*', ', '.join(columns), 1)
            
            # Format for consistency
            sql = sqlparse.format(sql, reindent=True, keyword_case='upper')
            return sql.rstrip(';')
        except:
            return sql
    
    def generate_sql(self, question: str) -> Tuple[str, Optional[str]]:
        """
        Generate SQL query from a natural language question after validating schema.
        
        Args:
            question: Natural language question
            
        Returns:
            Tuple of (generated_sql, error_message)
        """
        # Validate schema references
        question_lower = question.lower()
        words = re.findall(r'\b\w+\b', question_lower)
        schema_tables = set(table.lower() for table in self.schema_cache.keys())
        schema_columns = set(col['name'].lower() for table in self.schema_cache.values() 
                           for col in table)
        
        # Check for schema terms, including singular forms
        schema_terms = []
        for word in words:
            if word in schema_tables or word in schema_columns:
                schema_terms.append(word)
            else:
                singular = word.rstrip('s')
                if singular in schema_tables or singular in schema_columns:
                    schema_terms.append(singular)
        
        if not schema_terms:
            valid_tables = ", ".join(self.schema_cache.keys())
            return "", (f"No valid tables or columns referenced in the question. "
                       f"Available tables: {valid_tables}. Please clarify.")
        
        schema_prompt = self._format_schema_prompt()
        prompt = f"""Given the following database schema, translate the natural language question into a SQL query. Ensure the query uses valid table and column names from the schema. For exclusion queries (e.g., 'not play'), use NOT IN or LEFT JOIN with NULL checks.

Database Schema:
{schema_prompt}

Question: {question}
SQL Query:"""
        
        try:
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=256,
                    pad_token_id=self.tokenizer.eos_token_id,
                    num_beams=5,
                    temperature=0.7
                )
            
            generated_sql = self.tokenizer.decode(
                outputs[0][inputs["input_ids"].shape[1]:],
                skip_special_tokens=True
            ).strip()
            
            # Optimize the query
            generated_sql = self._optimize_query(generated_sql)
            
            # Validate generated SQL against schema
            sql_lower = generated_sql.lower()
            sql_tables = re.findall(r'\bfrom\s+(\w+)|join\s+(\w+)', sql_lower)
            sql_tables = [t for tup in sql_tables for t in tup if t]
            sql_columns = re.findall(r'\bselect\s+([\w\s,]+)\s+from', sql_lower)
            sql_columns = [c.strip() for cols in sql_columns for c in cols.split(',')] if sql_columns else []
            
            # Verify tables exist
            for table in sql_tables:
                if table not in schema_tables and table not in [t.lower() for t in self.schema_cache.keys()]:
                    valid_tables = ", ".join(self.schema_cache.keys())
                    return "", (f"Generated query references invalid table '{table}'. "
                               f"Available tables: {valid_tables}. Please clarify.")
            
            # For columns, we're more lenient because they might be using aggregates/functions
            
            return generated_sql, None
        except Exception as e:
            return "", f"Error generating SQL: {str(e)}"
    
    def execute_query(self, sql: str) -> Tuple[List, List, Optional[str]]:
        """
        Execute the SQL query on the database.
        
        Args:
            sql: SQL query to execute
            
        Returns:
            Tuple of (results, column_names, error_message)
        """
        try:
            # Debug the SQL query
            print(f"\nExecuting SQL query: {sql}")
            
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            # Fix case sensitivity issues with table and column names
            # Extract table names from schema
            actual_table_names = {table.lower(): table for table in self.schema_cache.keys()}
            
            # Replace table names in SQL with correct case
            modified_sql = sql
            for lower_name, actual_name in actual_table_names.items():
                # Use regex to find table names (after FROM or JOIN)
                modified_sql = re.sub(
                    fr'\bfrom\s+{lower_name}\b', 
                    f'FROM {actual_name}', 
                    modified_sql, 
                    flags=re.IGNORECASE
                )
                modified_sql = re.sub(
                    fr'\bjoin\s+{lower_name}\b', 
                    f'JOIN {actual_name}', 
                    modified_sql, 
                    flags=re.IGNORECASE
                )
                
                # Fix references to tables in column specifications (table.column)
                modified_sql = re.sub(
                    fr'\b{lower_name}\s*\.\s*', 
                    f'{actual_name}.', 
                    modified_sql, 
                    flags=re.IGNORECASE
                )
            
            # If the SQL was modified, log the change
            if modified_sql != sql:
                print(f"Modified SQL for case sensitivity: {modified_sql}")
                sql = modified_sql
            
            # Execute the query
            cursor.execute(sql)
            
            # Fetch results
            results = cursor.fetchall()
            
            # Get column names
            columns = [desc[0] for desc in cursor.description] if cursor.description else []
            
            # Debug the results
            print(f"Query returned {len(results)} rows with columns: {columns}")
            
            conn.close()
            return results, columns, None
        except sqlite3.OperationalError as e:
            error_msg = str(e)
            print(f"SQLite Error: {error_msg}")
            
            # Check if the error is due to a non-existent table or column
            if "no such table" in error_msg.lower() or "no such column" in error_msg.lower():
                valid_tables = ", ".join(self.schema_cache.keys())
                return [], [], (f"Schema error: {error_msg}. Available tables: {valid_tables}. "
                               "Please ensure the query references valid tables and columns.")
            return [], [], f"Error executing query: {error_msg}"
        except Exception as e:
            error_msg = str(e)
            print(f"Execution Error: {error_msg}")
            return [], [], f"Error executing query: {error_msg}"
    
    def run_query(self, question: str) -> Dict:
        """
        Run a complete query cycle: generate SQL, execute, and format results.
        Handles ambiguous queries by asking for clarification.
        Includes count of distinct results for queries requesting distinct values.
        
        Args:
            question: Natural language question
            
        Returns:
            Dictionary containing query results and metadata
        """
        # Check for ambiguity
        is_ambiguous, clarification_message = self._is_ambiguous(question)
        
        if is_ambiguous:
            print(f"Clarification needed: {clarification_message}")
            clarified_question = input("Please provide a clarified question: ").strip()
            
            if clarified_question.lower() in ['exit', 'quit']:
                return {"question": question, "sql": "", "results": [], "columns": [], "error": "User exited clarification"}
            
            if not clarified_question:
                return {"question": question, "sql": "", "results": [], "columns": [], "error": "No clarified question provided"}
            
            question = clarified_question
            
        print(f"\nProcessing question: {question}")
        
        # Generate SQL
        sql, error = self.generate_sql(question)
        if error:
            print(f"SQL generation error: {error}")
            return {"question": question, "sql": "", "results": [], "columns": [], "error": error}
        
        print(f"Generated SQL: {sql}")
        
        # Execute query
        results, columns, exec_error = self.execute_query(sql)
        if exec_error:
            print(f"Execution error: {exec_error}")
            return {"question": question, "sql": sql, "results": [], "columns": [], "error": exec_error}
        
        # Format results
        if results:
            # Convert any non-string values to strings for proper display
            str_results = [[str(cell) if cell is not None else 'NULL' for cell in row] 
                          for row in results]
            formatted_results = tabulate(str_results, headers=columns, tablefmt="grid")
            result_message = formatted_results
        else:
            result_message = "No results returned. This could mean the query found no matching data."
        
        # Check if the query involves DISTINCT and compute the count
        count_message = ""
        if "DISTINCT" in sql.upper() or "GROUP BY" in sql.upper():
            count = len(results) if results else 0
            count_message = f"\nNumber of distinct results: {count}"

        # Print the results to the console with clear formatting
        print("\n" + "="*50)
        print("QUERY RESULT:")
        print("="*50)
        print(result_message + count_message)
        print("="*50)

        return {
            "question": question,
            "sql": sql,
            "results": result_message + count_message,
            "columns": columns,
            "row_count": len(results),
            "error": None
        }
    
    def interactive_mode(self):
        """
        Run the agent in interactive mode with no limit on questions.
        Exits when user types 'exit' or 'quit'.
        """
        print("="*50)
        print("Text-to-SQL Agent Interactive Mode")
        print("="*50)
        print("Ask any number of questions about the database.")
        print("Database summary:")
        
        # Show table counts
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            for table in self.schema_cache.keys():
                cursor.execute(f"SELECT COUNT(*) FROM {table}")
                count = cursor.fetchone()[0]
                print(f"- {table}: {count} records")
            conn.close()
        except Exception as e:
            print(f"Error getting counts: {str(e)}")
            
        print("\nExample questions:")
        print("1. What is the name of the activity that has the most faculty members involved in?")
        print("2. How many students are majoring in each major?")
        print("3. Which faculty member advises the most students?")
        print("\nType 'exit' to quit.\n")
        
        question_count = 0
        while True:
            question_count += 1
            question = input(f"\nEnter question {question_count}: ").strip()
            
            if question.lower() in ['exit', 'quit']:
                print("\nExiting interactive mode...")
                break
                
            if not question:
                print("Please enter a valid question.")
                continue
                
            result = self.run_query(question)
            
            print("\n" + "-"*50)
            print(f"Question: {result['question']}")
            print(f"Generated SQL:\n{result['sql']}")
            
            if result['error']:
                print(f"Error: {result['error']}")
            else:
                print("\nResults:")
                print(result['results'])
                print(f"Row count: {result['row_count']}")
            
            print("-"*50 + "\n")
        
        print("Thank you for using the Text-to-SQL Agent!")
    
    def analyze_db(self):
        """
        Analyze the database to provide insights about the data.
        This helps diagnose issues with data retrieval.
        """
        print("\n" + "="*50)
        print("DATABASE ANALYSIS")
        print("="*50)
        
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            # Get all tables
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = [table[0] for table in cursor.fetchall()]
            
            for table in tables:
                # Get row count
                cursor.execute(f"SELECT COUNT(*) FROM {table}")
                row_count = cursor.fetchone()[0]
                
                # Get column info
                cursor.execute(f"PRAGMA table_info({table});")
                columns = cursor.fetchall()
                
                print(f"\nTable: {table}")
                print(f"Row count: {row_count}")
                print("Columns:")
                for col in columns:
                    print(f"  - {col[1]} ({col[2]})")
                
                # Get sample data
                if row_count > 0:
                    cursor.execute(f"SELECT * FROM {table} LIMIT 3")
                    samples = cursor.fetchall()
                    print("Sample data:")
                    for i, sample in enumerate(samples):
                        print(f"  Row {i+1}: {sample}")
            
            conn.close()
            print("="*50)
        except Exception as e:
            print(f"Error analyzing database: {str(e)}")
            print("="*50)

# Example usage:
if __name__ == "__main__":
    # Initialize the agent with paths to your model and database
    model_path = "llama3-text2sql"  # Path to your model directory
    db_path = "spider_sqlite/activity_1/activity_1.sqlite"  # Path to your SQLite database
    
    agent = Text2SQLAgent(
        model_path=model_path,
        db_path=db_path,
        populate_if_empty=True  # Automatically populate the database if empty
    )
    
    # Run database analysis to understand the schema better
    agent.analyze_db()
    
    # Start interactive mode
    agent.interactive_mode()

2025-05-11 01:20:33,027 - INFO - Based on the current allocation process, no modules could be assigned to the following devices due to insufficient memory:
  - 0: 2101346304 bytes required
These minimum requirements are specific to this allocation attempt and may vary. Consider increasing the available memory for these devices to at least the specified minimum, or adjusting the model config.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-05-11 01:20:50,963 - INFO - Based on the current allocation process, no modules could be assigned to the following devices due to insufficient memory:
  - 0: 2101346304 bytes required
These minimum requirements are specific to this allocation attempt and may vary. Consider increasing the available memory for these devices to at least the specified minimum, or adjusting the model config.


Table 'activity' columns: activity_id, activity_name
Row count: 0
Table 'participates_in' columns: student_id, activity_id
Row count: 0
Table 'faculty_participates_in' columns: faculty_id, activity_id
Row count: 0
Table 'student' columns: student_id, last_name, first_name, age, sex, major, advisor, city_code
Row count: 0
Table 'faculty' columns: faculty_id, last_name, first_name, rank, sex, phone, room, building
Row count: 0

--- DATABASE IS EMPTY. POPULATING WITH SAMPLE DATA ---

Inserted 15 activities
Inserted 50 faculty members
Inserted 200 students
Inserted 88 faculty activity associations
Inserted 393 student activity associations

--- DATABASE POPULATION COMPLETE ---

Table 'activity' columns: activity_id, activity_name
Row count: 15
Sample data: (1, 'Chess Club')
Table 'participates_in' columns: student_id, activity_id
Row count: 393
Sample data: (1, 9)
Table 'faculty_participates_in' columns: faculty_id, activity_id
Row count: 88
Sample data: (49, 1)
Table 'student' columns: st


Enter question 1:  What is the name of the activity that has the most faculty members involved in?


...

Clarification needed: The question is vague. Please specify which fields or conditions you want to query.


Please provide a clarified question:  Show me the name and age of students who are majoring in 101



Processing question: Show me the name and age of students who are majoring in 101
Generated SQL: SELECT T1.last_name,
       T1.age
FROM student AS T1
JOIN participates_in AS T2 ON T1.student_id = T2.student_id
WHERE T1.major = 101
UNION
SELECT T1.last_name,
       T1.age
FROM student AS T1
JOIN participates_in AS T2 ON T1.student_id = T2.student_id
WHERE T1.major = 102
UNION
SELECT T1.last_name,
       T1.age
FROM student AS T1
JOIN participates_in AS T2 ON T1.student_id = T2.student_id
WHERE T1.major = 103
UNION
SELECT T1.last_name,
       T1.age
FROM student AS T1
JOIN participates_in AS T2 ON T1.student_id = T2.student_id
WHERE T1.major = 104
UNION
SELECT T1.last_name,
       T1.age
FROM student AS T1
JOIN participates_in AS T2 ON T1.student_id = T2.student_id
WHERE T1.major = 105
UNION
SELECT T1.last_name,
       T1.age
FROM student AS T1
JOIN participates_in AS T2 ON T1.student_id = T2.student_id
WHERE T1.major =

Executing SQL query: SELECT T1.last_name,
       T1.age
FROM stud


Enter question 3:  EXIT



Exiting interactive mode...
Thank you for using the Text-to-SQL Agent!
